<a href="https://colab.research.google.com/github/agg-shambhavi/Glasses-Removal-Network/blob/main/GlassesRemovalNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Constants 

In [ ]:
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 1
LEARNING_RATE = 2e-4
NUM_EPOCHS = 2
LAMBDA_IDENTITY = 0.0
LAMBDA_CYCLE = 10
# Korean paper trained for 200 epochs

# File Paths

In [ ]:
import zipfile
import os

In [ ]:
def extract_zip_file(path):
  # extract all contents of zip file
  zip_train = zipfile.ZipFile(path, 'r')
  zip_train.extractall("./")
  zip_train.close()

In [ ]:
trainGlassesZip  = "/content/drive/MyDrive/Computer Vision /Cyclegan/Data/train glasses.zip"
trainNoglassesZip = "/content/drive/MyDrive/Computer Vision /Cyclegan/Data/train non glasses.zip"
valGlassesZip  = "/content/drive/MyDrive/Computer Vision /Cyclegan/Data/val glasses.zip"
valNoglassesZip = "/content/drive/MyDrive/Computer Vision /Cyclegan/Data/val non glasses.zip"

In [ ]:
extract_zip_file(trainGlassesZip)
extract_zip_file(trainNoglassesZip)
extract_zip_file(valGlassesZip)
extract_zip_file(valNoglassesZip)

In [ ]:
trainGlassesDir = "/content/train glasses"
trainNoglassesDir = "/content/train non glasses"
valGlassesDir = "/content/val glasses"
valNoglassesDir = "/content/val non glasses"

In [ ]:
resultsNoGlasses = "/content/drive/MyDrive/Computer Vision /Cyclegan/Results - G -> N"
resultGlasses = "/content/drive/MyDrive/Computer Vision /Cyclegan/Results N -> G "

# Imports

In [ ]:
!pip install -U git+https://github.com/albu/albumentations > /dev/null 

  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-pajr_r11


In [ ]:
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
from PIL import Image, ImageOps
import numpy as np
from torch.utils.data import Dataset
from torchvision.utils import save_image
import torchvision.models.vgg as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import time

# Dataset Class & Transforms

In [ ]:
class GlassesDataset(Dataset):
    def __init__(self, root_dir_glass, root_dir_noglass, transform=None):
        super().__init__()
        self.root_dir_glasses = root_dir_glass
        self.root_dir_noglasses = root_dir_noglass
        self.list_glasses = os.listdir(self.root_dir_glasses)
        self.list_noglasses = os.listdir(self.root_dir_noglasses)
        self.transform = transform
        self.len_glasses = len(self.list_glasses)
        self.len_noglasses = len(self.list_noglasses)
        self.dataset_length = max(self.len_glasses, self.len_noglasses)
        
    def __len__(self):
        return self.dataset_length
    
    def __getitem__(self, index):
        start_time = time.time()
        glasses_file = self.list_glasses[index % self.len_glasses]
        no_glasses_file = self.list_noglasses[index % self.len_noglasses]
        glasses_path = os.path.join(self.root_dir_glasses, glasses_file)
        no_glasses_path = os.path.join(self.root_dir_noglasses, no_glasses_file)
        img_glasses = np.array(ImageOps.grayscale(Image.open(glasses_path).convert('RGB')))
        img_noglasses = np.array(ImageOps.grayscale(Image.open(no_glasses_path).convert('RGB')))
        
        if self.transform:
            augmentation = self.transform(image=img_glasses, image0=img_noglasses)
            img_glasses = augmentation["image"]
            img_noglasses = augmentation["image0"]

        end_time = time.time()
        print(f"{end_time - start_time} data loading time")
            
        return img_glasses, img_noglasses

In [ ]:
transforms_train = A.Compose(
    [
        A.Resize(width=256, height=256),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5], std=[0.5], max_pixel_value=255),
        ToTensorV2(),
     ],
    additional_targets={"image0": "image"},
)

In [ ]:
transforms_val = A.Compose(
    [
        A.Resize(width=256, height=256),
        A.Normalize(mean=[0.5], std=[0.5], max_pixel_value=255),
        ToTensorV2(),
     ],
    additional_targets={"image0": "image"},
)

# Model Architecture 
## Generator

In [ ]:
class SelfAttentionBlock(nn.Module):
    """Self Attention Layer/Block"""

    def __init__(self, in_dim):
        super().__init__()
        self.in_channel = in_dim
        self.query_conv = nn.Conv2d(
            in_channels=in_dim, out_channels=in_dim // 8, kernel_size=1
        )
        self.key_conv = nn.Conv2d(
            in_channels=in_dim, out_channels=in_dim // 8, kernel_size=1
        )
        self.value_conv = nn.Conv2d(
            in_channels=in_dim, out_channels=in_dim, kernel_size=1
        )
        self.gamma = nn.Parameter(torch.zeros(1))
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        """
        inputs:
            x : input feature maps (B X C X W X H)
        returns:
            out : self attention value + input features
            attention: B x N X N (N is Width*Height)
        """

        batch_size, c, h, w = x.shape
        proj_query = self.query_conv(x).view(batch_size, -1, w * h).permute(0, 2, 1)
        # proj_query with view is of shape (batch_size, c, w*h)
        # proj_query with permute becomes batch_size, w*h, c), ie transpose of proj_query for matrix multiplication
        proj_key = self.key_conv(x).view(batch_size, -1, w * h)  # (batch_size, c, w*h)

        energy = torch.bmm(proj_query, proj_key)  # (batch_size, w*h, w*h)
        attention = self.softmax(energy)  # (batch_size, w*h, w*h)

        proj_value = self.value_conv(x).view(
            batch_size, -1, w * h
        )  # (batch_size, c*8, w*h) c*8 = C

        out = torch.bmm(
            proj_value, attention.permute(0, 2, 1)
        )  # (batch_size, c*8, w*h)
        # attention is transposed for matrix mul
        out = out.view(batch_size, c, w, h)

        out = self.gamma * out + x
        return out, attention


class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            # I am not sure which res block to take
            nn.Conv2d(
                in_channels=channels,
                out_channels=channels,
                kernel_size=1,
                stride=1,
                padding=0,
            )
            # nn.Conv2d(
            #     in_channels=channels,
            #     out_channels=channels,
            #     kernel_size=7,
            #     stride=1,
            #     padding=3,
            # )
        )

    def forward(self, x):
        return x + self.block(x)


class Generator(nn.Module):
    def __init__(self, img_channels):
        super().__init__()

        self.downConvBlock = nn.Sequential(
            nn.Conv2d(
                in_channels=img_channels,
                out_channels=1,
                kernel_size=7,
                stride=1,
                padding=3,
            ),
            nn.Conv2d(
                in_channels=1,
                out_channels=64,
                kernel_size=6,
                stride=2,
                padding=2,
                bias=False,
            ),
            nn.InstanceNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=2,
                padding=1,
                bias=False,
            ),
            nn.InstanceNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=128,
                out_channels=256,
                kernel_size=3,
                stride=2,
                padding=1,
                bias=False,
            ),
            nn.InstanceNorm2d(256),
            nn.ReLU(),
        )

        self.convtrans1 = nn.ConvTranspose2d(
            in_channels=256,
            out_channels=128,
            kernel_size=3,
            stride=2,
            padding=1,
            bias=False,
        )

        self.active1 = nn.Sequential(nn.InstanceNorm2d(128), nn.ReLU())
        self.convtrans2 = nn.ConvTranspose2d(
            in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1
        )
        self.convtrans3 = nn.ConvTranspose2d(
            in_channels=64, out_channels=1, kernel_size=6, stride=2, padding=2
        )
        self.selfAttentionlayer = SelfAttentionBlock(in_dim=256)
        self.resBlock1 = ResidualBlock(channels=256)
        self.resBlock2 = ResidualBlock(channels=256)

    def forward(self, x):
        batch_size, ch, h, w = x.shape
        out = self.downConvBlock(x)
        out = self.resBlock1(out)
        out, _ = self.selfAttentionlayer(out)
        out = self.resBlock2(out)
        out = self.convtrans1(out, output_size=(batch_size, 128, 32, 32))
        out = self.active1(out)
        out = self.convtrans2(out, output_size=(batch_size, 64, 64, 64))
        out = self.convtrans3(out, output_size=(batch_size, 1, 128, 128))
        out = self.lastlayer(out)
        return out

## Discriminator

In [ ]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=4,
                stride=stride,
                padding=1,
                bias=True,
                padding_mode="reflect",
            ),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2),
        )

        layers = []

        in_channels = features[0]

        for feature in features[1:]:
            layers.append(
                Block(in_channels, feature, stride=1 if feature == features[-1] else 2)
            )
            in_channels = feature

        layers.append(
            nn.Conv2d(
                in_channels,
                1,
                kernel_size=4,
                stride=1,
                padding=1,
                padding_mode="reflect",
            )
        )

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        return torch.sigmoid(self.model(x))

# Utility Functions

In [ ]:
def save_some_examples_g(gen, val_loader, epoch, idx, folder):
    img_glasses, img_noglasses = next(iter(val_loader))
    img_glasses, img_noglasses = img_glasses.to(DEVICE), img_noglasses.to(DEVICE)
    gen.eval()
    with torch.no_grad():
        y_fake_glasses = gen(img_noglasses)
        y_fake_glasses = y_fake_glasses * 0.5 + 0.5
        save_image(y_fake_glasses, os.path.join(folder, f"{epoch}_{idx}_fake_glasses.png"))
        # save_image(img_noglasses, os.path.join(folder, f"{epoch}_no_glasses.png"))
        # save_image(img_glasses, os.path.join(folder, f"{epoch}_glasses.png"))
    gen.train()

def save_some_examples_n(gen, val_loader, epoch, idx, folder):
    img_glasses, img_noglasses = next(iter(val_loader))
    img_glasses, img_noglasses = img_glasses.to(DEVICE), img_noglasses.to(DEVICE)
    gen.eval()
    with torch.no_grad():
        y_fake_no_glasses = gen(img_glasses)
        y_fake_no_glasses = y_fake_no_glasses * 0.5 + 0.5
        save_image(y_fake_no_glasses, os.path.join(folder, f"{epoch}_{idx}_fake_no_glasses.png"))
        # save_image(img_glasses, os.path.join(folder, f"{epoch}_glasses.png"))
        # save_image(img_noglasses, os.path.join(folder, f"{epoch}_no_glasses.png"))
    gen.train()


def save_checkpoint(model, optimizer, epoch, filename):
    filename = str(epoch) + filename + "_cpt.pth.tar"
    print("=> Saving Checkpoint")
    checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading Checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state(checkpoint["state_dict"])
    optimizer.load_state(checkpoint["state_dict"])
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


# Train Function

In [ ]:
def getVggFeatures(input, i):
    # function to get vgg features
    # phi according to research paper
    # i means the same as that of the paper
    layer_dict = {1: 5, 2: 10, 3: 17, 4: 24, 5: 30}
    vgg16 = models.vgg16(pretrained=True)
    output = vgg16.features[: layer_dict[i]](input)
    return output


def pcclT1(cycle_specs, img_glasses, l1):
    # need to convert b/w into 3 channels
    batch_size, c, w, h = cycle_specs.shape
    cycle_specs = cycle_specs.repeat(batch_size, 3, w, h)
    img_glasses = img_glasses.repeat(batch_size, 3, w, h)
    out = 0
    for i in range(1, 6):
        phi_gened = getVggFeatures(cycle_specs, i)
        phi_og = getVggFeatures(img_glasses, i)
        out += l1(phi_gened - phi_og)
    # i dont know if out has to be divided by 5 or not
    return out


def pcclT2(cycle_nospecs, img_noglasses, l1):
    # need to convert b/w into 3 channels
    batch_size, c, w, h = cycle_nospecs.shape
    out = 0
    cycle_nospecs = cycle_nospecs.repeat(batch_size, 3, w, h)
    img_noglasses = img_noglasses.repeat(batch_size, 3, w, h)
    out = 0
    for i in range(1, 6):
        phi_gened = getVggFeatures(cycle_nospecs, i)
        phi_og = getVggFeatures(img_noglasses, i)
        out += l1(phi_gened - phi_og)
    # i dont know if out has to be divided by 5 or not
    return out


def trainFn(
    disc_G, disc_N, gen_G, gen_N, loader, optim_g, optim_d, l1, mse, val_loader, epoch
):
    loop = tqdm(loader, leave=True, position=0)

    for idx, (img_glasses, img_noglasses) in enumerate(loop):
        img_glasses = img_glasses.to(DEVICE)
        img_noglasses = img_noglasses.to(DEVICE)

        # Train Discriminators
        start_time = time.time()

        fake_nospecs = gen_N(img_glasses)
        disc_ns_real = disc_N(img_noglasses)
        disc_ns_fake = disc_N(fake_nospecs.detach())
        disc_ns_real_loss = mse(disc_ns_real, torch.ones_like(disc_ns_real))
        disc_ns_fake_loss = mse(disc_ns_fake, torch.zeros_like(disc_ns_fake))
        disc_ns_loss = disc_ns_real_loss + disc_ns_fake_loss

        fake_specs = gen_G(img_noglasses)
        disc_s_real = disc_G(img_glasses)
        disc_s_fake = disc_G(fake_specs.detach())
        disc_s_real_loss = mse(disc_s_real, torch.ones_like(disc_s_real))
        disc_s_fake_loss = mse(disc_s_fake, torch.zeros_like(disc_s_fake))
        disc_s_loss = disc_s_real_loss + disc_s_fake_loss

        end_time = time.time()
        print(f"{end_time - start_time} Discriminators training")

        # grand total
        D_loss = (disc_ns_loss + disc_s_loss) / 2

        # backprop and update weights of discriminator

        start_time = time.time()

        optim_d.zero_grad()
        D_loss.backward()
        optim_d.step()

        end_time = time.time()
        print(f"{end_time - start_time} backprop discriminator")

        # Train Generators
        # adversarial loss

        start_time = time.time()

        disc_ns_fake = disc_N(fake_nospecs)
        disc_s_fake = disc_G(fake_specs)
        loss_g_s = mse(disc_s_fake, torch.ones_like(disc_s_fake))
        loss_g_ns = mse(disc_ns_fake, torch.ones_like(disc_ns_fake))

        end_time = time.time()
        print(f"{end_time - start_time} adversarial loss")

        # cycle loss

        start_time = time.time()

        cycle_specs = gen_G(fake_nospecs)
        cycle_nospecs = gen_N(fake_specs)
        specs_cycle_loss = l1(img_glasses, cycle_specs)
        nospecs_cycle_loss = l1(img_noglasses, cycle_nospecs)

        end_time = time.time()
        print(f"{end_time - start_time} cycle loss")

        ## Perceptual Loss
        start_time = time.time()
        loss_pccl = pcclT1(cycle_specs, img_glasses, l1) + pcclT2(
            cycle_nospecs, img_noglasses, l1
        )
        end_time = time.time()
        print(f"{end_time - start_time} Perceptual loss")

        # grand total
        G_loss = (
            loss_g_s
            + loss_g_ns
            + (specs_cycle_loss * LAMBDA_CYCLE)
            + (nospecs_cycle_loss * LAMBDA_CYCLE)
            + loss_pccl
        )

        # backprop and update weights of discriminator

        start_time = time.time()

        optim_g.zero_grad()
        G_loss.backward()
        optim_g.step()

        end_time = time.time()
        print(f"{end_time - start_time} generator backprop")

        # if idx == 2000:
        #   print(f"Generator Loss: {G_loss} Discriminator Loss: {D_loss}")

        # if idx % 2 == 0:
        print(f"Generator Loss: {G_loss} Discriminator Loss: {D_loss}")

        save_some_examples_g(gen_G, val_loader, epoch, idx, folder=resultGlasses)
        save_some_examples_n(gen_N, val_loader, epoch, idx, folder=resultsNoGlasses)


# Main Function

In [ ]:
def main():
  # Instantiate Generator and Discriminators 
  disc_G = Discriminator(in_channels=1).to(DEVICE)
  disc_N = Discriminator(in_channels=1).to(DEVICE)
  gen_G = Generator(img_channels=1, num_residuals=4).to(DEVICE)
  gen_N = Generator(img_channels=1, num_residuals=4).to(DEVICE)

  # Initialize Optimizers
  opt_disc = optim.Adam(
      list(disc_G.parameters()) + list(disc_N.parameters()),
      lr = LEARNING_RATE,
      betas = (0.5, 0.999)
  )

  opt_gen = optim.Adam(
      list(gen_G.parameters()) + list(gen_N.parameters()),
      lr = LEARNING_RATE,
      betas = (0.5, 0.999)
  )

  # Loss Function
  L1 = nn.L1Loss()
  mse = nn.MSELoss()

  # Instantiate Datasets
  trainData = GlassesDataset(root_dir_glass=trainGlassesDir, root_dir_noglass=trainNoglassesDir, transform=transforms_train)
  valData = GlassesDataset(root_dir_glass=valGlassesDir, root_dir_noglass=valNoglassesDir, transform=transforms_val)

  # Dataloader
  train_loader = DataLoader(trainData, BATCH_SIZE, shuffle=True)
  val_loader = DataLoader(valData, BATCH_SIZE, shuffle=False)

  # TRAINING LOOP
  for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch}")
    trainFn(disc_G, disc_N, gen_G, gen_N, train_loader, opt_gen, opt_disc, L1, mse, val_loader, epoch)

    # if epoch % 3 == 0:
    save_checkpoint(disc_G, opt_disc, epoch, "disc_G")
    save_checkpoint(disc_N, opt_disc, epoch, "disc_N")
    save_checkpoint(gen_G, opt_gen, epoch, "gen_G")
    save_checkpoint(gen_N, opt_gen, epoch, "gen_N")

    # save_some_examples_g(gen_G, val_loader, epoch, folder = resultGlasses)
    # save_some_examples_n(gen_N, val_loader, epoch, folder = resultsNoGlasses)

In [ ]:
main()

  0%|          | 0/2035 [00:00<?, ?it/s]

Epoch 0
0.19081783294677734 data loading time
3.1997427940368652 Discriminators training
1.1547625064849854 backprop discriminator
0.24422144889831543 adversarial loss
2.3628146648406982 cycle loss
9.593616962432861 generator backprop
Generator Loss: 9.346444129943848 Discriminator Loss: 0.5261926651000977
0.11168742179870605 data loading time
0.11684632301330566 data loading time


  0%|          | 1/2035 [00:19<10:50:50, 19.20s/it]

0.11888289451599121 data loading time
2.5681607723236084 Discriminators training
1.098221778869629 backprop discriminator
0.2455005645751953 adversarial loss
2.1605730056762695 cycle loss
9.403878211975098 generator backprop
Generator Loss: 10.514144897460938 Discriminator Loss: 0.5621699690818787
0.11118292808532715 data loading time
0.11231112480163574 data loading time


  0%|          | 2/2035 [00:37<10:38:04, 18.83s/it]

...

0.13042616844177246 data loading time
2.6372077465057373 Discriminators training
1.1314878463745117 backprop discriminator
0.2498931884765625 adversarial loss
2.1389124393463135 cycle loss
9.608829259872437 generator backprop
Generator Loss: 2.22938871383667 Discriminator Loss: 0.4781583249568939
0.11560845375061035 data loading time
0.11382770538330078 data loading time


 43%|████▎     | 872/2035 [4:20:53<5:54:08, 18.27s/it]

0.13698840141296387 data loading time
2.637333631515503 Discriminators training
1.1336638927459717 backprop discriminator
0.3357870578765869 adversarial loss
2.1535727977752686 cycle loss
9.746977806091309 generator backprop
Generator Loss: 2.459934711456299 Discriminator Loss: 0.4059765338897705
0.11156702041625977 data loading time
0.11460328102111816 data loading time


 43%|████▎     | 873/2035 [4:21:12<5:55:45, 18.37s/it]

0.13213014602661133 data loading time
2.6283962726593018 Discriminators training
1.120863914489746 backprop discriminator
0.24959492683410645 adversarial loss
2.1492292881011963 cycle loss
